# Intent Resolution Evaluator

The Intent Resolution evaluator measures how well an agent has identified and resolved the user intent.
The scoring is on a 1-5 integer scale and is as follows:

  - Score 1: Response completely unrelated to user intent
  - Score 2: Response minimally relates to user intent
  - Score 3: Response partially addresses the user intent but lacks complete details
  - Score 4: Response addresses the user intent with moderate accuracy but has minor inaccuracies or omissions
  - Score 5: Response directly addresses the user intent and fully resolves it

The evaluation requires the following inputs:

  - Query    : The user query. Either a string with a user request or a list of messages with previous requests from the user and responses from the assistant, potentially including a system message.
  - Response : The response to be evaluated. Either a string or a message with the response from the agent to the last user query.

There is a third optional parameter:
  - ToolDefinitions : The list of tool definitions the agent can call. This may be useful for the evaluator to better assess if the right tool was called to resolve a given intent.

### Initialize Intent Resolution Evaluator


In [10]:
import os

from azure.ai.projects import AIProjectClient
from azure.ai.projects.models import ConnectionType
from azure.identity import DefaultAzureCredential
from azure.ai.evaluation import IntentResolutionEvaluator
from dotenv import load_dotenv
from pprint import pprint

#load_dotenv() loads the PROJECT_CONNECTION_STRING, MODEL_DEPLOYMENT_NAME and MODEL_DEPLOYMENT_API_VERSION variables
#from a .env file in the current directory. You can also set these in your environment directly if you prefer
load_dotenv()

project_client = AIProjectClient.from_connection_string(
    credential=DefaultAzureCredential(),
    conn_str=os.environ["PROJECT_CONNECTION_STRING"],
)
model_config = project_client.connections.get_default(
                                            connection_type=ConnectionType.AZURE_OPEN_AI,
                                            include_credentials=True) \
                                         .to_evaluator_model_config(
                                            deployment_name=os.environ["MODEL_DEPLOYMENT_NAME"],
                                            api_version=os.environ["MODEL_DEPLOYMENT_API_VERSION"],
                                            include_credentials=True
                                          )

intent_resolution_evaluator = IntentResolutionEvaluator(model_config)

### Samples

#### Evaluating query and response as string

In [16]:
#Success example. Intent is identified and understood and the response correctly resolves user intent
result = intent_resolution_evaluator(query="What are the opening hours of the Eiffel Tower?",
                                     response="Opening hours of the Eiffel Tower are 9:00 AM to 11:00 PM.",
                                    )
pprint(result)

{'additional_details': {'actual_user_intent': 'find out the opening hours of '
                                              'the Eiffel Tower',
                        'agent_perceived_intent': 'provide the opening hours '
                                                  'of the Eiffel Tower',
                        'conversation_has_intent': True,
                        'correct_intent_detected': True,
                        'intent_resolved': True},
 'intent_resolution': 5.0,
 'intent_resolution_reason': 'The response provides the opening hours of the '
                             'Eiffel Tower, which directly addresses the '
                             "user's query with accurate and complete "
                             'information. No additional details or tools were '
                             'required to resolve the query.',
 'intent_resolution_result': 'pass',
 'intent_resolution_threshold': 5}


In [ ]:
#Failure example. Even though intent is correctly identified, the response does not resolve the user intent
result = intent_resolution_evaluator(query="What is the opening hours of the Eiffel Tower?",
                                     response="Please check the official website for the up-to-date information on Eiffel Tower opening hours.",
                                    )
pprint(result)

{'additional_details': {'actual_user_intent': 'find out the opening hours of '
                                              'the Eiffel Tower',
                        'agent_perceived_intent': 'provide information about '
                                                  'Eiffel Tower opening hours',
                        'conversation_has_intent': True,
                        'correct_intent_detected': True,
                        'intent_resolved': False},
 'intent_resolution': 2.0,
 'intent_resolution_reason': 'The response suggests checking the official '
                             'website for up-to-date information, which is a '
                             'generic and minimally helpful answer. While it '
                             'acknowledges the topic of the query (Eiffel '
                             'Tower opening hours), it does not provide any '
                             'specific or actionable information, leaving the '
                             'query 

#### Evaluating query and response as list of messages

In [ ]:
query = [
    {
        "role": "system",
        "content": "You are a friendly and helpful customer service agent."
    },
    {
        "createdAt": "2025-03-14T06:14:20Z",
        "role": "user",
        "content": [
            {
                "type": "text",
                "text": "Hi, I need help with the last 2 orders on my account #888. Could you please update me on their status?"
            }
        ]
    }
]

response = [
    {
        "createdAt": "2025-03-14T06:14:30Z",
        "run_id": "0",
        "role": "assistant",
        "content": [
            {
                "type": "text",
                "text": "Hello! Let me quickly look up your account details."
            }
        ]
    },
    {
        "createdAt": "2025-03-14T06:14:35Z",
        "run_id": "0",
        "role": "assistant",
        "content": [
            {
                "type": "tool_call",
                "tool_call": {
                    "id": "tool_call_20250310_001",
                    "type": "function",
                    "function": {
                        "name": "get_orders",
                        "arguments": {
                            "account_number": "888"
                        }
                    }
                }
            }
        ]
    },
    {
        "createdAt": "2025-03-14T06:14:40Z",
        "run_id": "0",
        "tool_call_id": "tool_call_20250310_001",
        "role": "tool",
        "content": [
            {
                "type": "tool_result",
                "tool_result": "[{ \"order_id\": \"123\" }, { \"order_id\": \"124\" }]"
            }
        ]
    },
    {
        "createdAt": "2025-03-14T06:14:45Z",
        "run_id": "0",
        "role": "assistant",
        "content": [
            {
                "type": "text",
                "text": "Thanks for your patience. I see two orders on your account. Let me fetch the details for both."
            }
        ]
    },
    {
        "createdAt": "2025-03-14T06:14:50Z",
        "run_id": "0",
        "role": "assistant",
        "content": [
            {
                "type": "tool_call",
                "tool_call": {
                    "id": "tool_call_20250310_002",
                    "type": "function",
                    "function": {
                        "name": "get_order",
                        "arguments": {
                            "order_id": "123"
                        }
                    }
                }
            },
            {
                "type": "tool_call",
                "tool_call": {
                    "id": "tool_call_20250310_003",
                    "type": "function",
                    "function": {
                        "name": "get_order",
                        "arguments": {
                            "order_id": "124"
                        }
                    }
                }
            }
        ]
    },
    {
        "createdAt": "2025-03-14T06:14:55Z",
        "run_id": "0",
        "tool_call_id": "tool_call_20250310_002",
        "role": "tool",
        "content": [
            {
                "type": "tool_result",
                "tool_result": "{ \"order\": { \"id\": \"123\", \"status\": \"shipped\", \"delivery_date\": \"2025-03-15\" } }"
            }
        ]
    },
    {
        "createdAt": "2025-03-14T06:15:00Z",
        "run_id": "0",
        "tool_call_id": "tool_call_20250310_003",
        "role": "tool",
        "content": [
            {
                "type": "tool_result",
                "tool_result": "{ \"order\": { \"id\": \"124\", \"status\": \"delayed\", \"expected_delivery\": \"2025-03-20\" } }"
            }
        ]
    },
    {
        "createdAt": "2025-03-14T06:15:05Z",
        "run_id": "0",
        "role": "assistant",
        "content": [
            {
                "type": "text",
                "text": "The order with ID 123 has been shipped and is expected to be delivered on March 15, 2025. However, the order with ID 124 is delayed and should now arrive by March 20, 2025. Is there anything else I can help you with?"
            }
        ]
    }
]

#please note that the tool definitions are not strictly required, and that some of the tools below are not used in the example above and that is ok.
#if context length is a concern you can remove the unused tool definitions or even the tool definitions altogether as the impact to the intent resolution evaluation is usual minimal.
tool_definitions = [
    {
        "name": "get_orders",
        "description": "Get the list of orders for a given account number.",
        "parameters": {
            "type": "object",
            "properties": {
                "account_number": {
                    "type": "string",
                    "description": "The account number to get the orders for."
                }
            }
        }
    },
    {
        "name": "get_order",
        "description": "Get the details of a specific order.",
        "parameters": {
            "type": "object",
            "properties": {
                "order_id": {
                    "type": "string",
                    "description": "The order ID to get the details for."
                }
            }
        }
    },
    {
        "name": "initiate_return",
        "description": "Initiate the return process for an order.",
        "parameters": {
            "type": "object",
            "properties": {
                "order_id": {
                    "type": "string",
                    "description": "The order ID for the return process."
                }
            }
        }
    },
    {
        "name": "update_shipping_address",
        "description": "Update the shipping address for a given account.",
        "parameters": {
            "type": "object",
            "properties": {
                "account_number": {
                    "type": "string",
                    "description": "The account number to update."
                },
                "new_address": {
                    "type": "string",
                    "description": "The new shipping address."
                }
            }
        }
    }
]

result = intent_resolution_evaluator(query            = query,
                                     response         = response,
                                     tool_definitions = tool_definitions,
                                    )
pprint(result)

{'additional_details': {'actual_user_intent': 'get an update on the status of '
                                              'the last two orders for account '
                                              '#888',
                        'agent_perceived_intent': 'provide an update on the '
                                                  'status of the last two '
                                                  'orders for account #888',
                        'conversation_has_intent': True,
                        'correct_intent_detected': True,
                        'intent_resolved': True},
 'intent_resolution': 5.0,
 'intent_resolution_reason': "The assistant accurately understood the user's "
                             'intent to get an update on the status of their '
                             'last two orders and fully resolved the query by '
                             'retrieving the relevant order details using '
                             'tools and providing

### Evaluating an agent conversation loaded from disk

In [27]:
import json
from azure.ai.evaluation import AIAgentConverter

def load_conversations(filename):
    with open(filename, 'r') as file:
        lines = file.readlines()
        parsed_conversations = [json.loads(line) for line in lines]
    print(f"Loaded {len(parsed_conversations)} conversations from {filename}.")
    return parsed_conversations

conversations_filename = r'sample_synthetic_conversations.jsonl'

#this loads 90 conversations from the file sample_synthetic_conversations.jsonl
sample_conversations = load_conversations(conversations_filename)

#get the first conversation from the loaded conversations
conversation = sample_conversations[10]

run_ids = AIAgentConverter.run_ids_from_conversation(conversation)
print(f"Run IDs in conversation: {run_ids}")
run_id = str(run_ids[0]) # convert runid to string in case it is some other type, e.g. an int
converted_conv = AIAgentConverter.convert_from_conversation(conversation, run_id)
# Extract the query and response from the conversation
query = converted_conv['query']
response = converted_conv['response']
tool_definitions = converted_conv['tool_definitions']

print(f"Run ID: {run_id}")
print(f"Query: {query}")
print(f"Response: {response}")
print(f"Tool Definitions: {tool_definitions}")

result = intent_resolution_evaluator(query = query, response = response, tool_definitions = tool_definitions)
print(f"Evaluation result")
pprint(result)

Loaded 90 conversations from sample_synthetic_conversations.jsonl.
Run IDs in conversation: [0, 1, 2]
Run ID: 0
Query: [{'role': 'system', 'content': 'You are a healthcare support agent assisting patients with appointment scheduling, prescription refills, test results, and general health inquiries.'}, {'createdAt': 1741618732, 'role': 'user', 'content': [{'type': 'text', 'text': 'Can you update my health records? I recently had a lab test and need the results added to my profile.'}]}]
Response: [{'createdAt': 1741618737, 'run_id': '0', 'role': 'assistant', 'content': [{'type': 'text', 'text': 'I’ll update your health records with the new lab test results. One moment, please.'}]}]
Tool Definitions: [{'name': 'schedule_appointment', 'description': 'Schedule an appointment for a patient.', 'parameters': {'type': 'object', 'properties': {'patient_id': {'type': 'string', 'description': 'The patient’s unique ID.'}, 'doctor': {'type': 'string', 'description': 'The doctor to schedule with.'}, 

# Putting it all together and evaluate an entire conversation run by run

In [30]:
def evaluate_conversation_run(conversation : dict, run_id : str, verbose=False):
    converted_conv = AIAgentConverter.convert_from_conversation(conversation, str(run_id))
    # Extract the query and response from the conversation
    query = converted_conv['query']
    response = converted_conv['response']
    tool_definitions = converted_conv['tool_definitions']
    
    if verbose:
        print(f"*********************************************")
        print(f"Evaluating conversation run with ID: {run_id}")
        print(f"Run ID: {run_id}")
        print(f"Query: {query}")
        print(f"Response: {response}")
        print(f"Tool Definitions: {tool_definitions}")

    # Evaluate the query and response using the intent resolution evaluator
    evaluation_result = intent_resolution_evaluator(query = query, response = response, tool_definitions = tool_definitions)
    if verbose:
        print(f"Evaluation Result:")
        pprint(evaluation_result)

    return evaluation_result

def evaluate_conversation(conversation, verbose=False):
    run_ids = AIAgentConverter.run_ids_from_conversation(conversation)
    print(f"Runs available in conversation: {run_ids}")
    results = []
    for run_id in run_ids:
        result = evaluate_conversation_run(conversation, str(run_id), verbose)
        results.append(result)
    return results

sample_conversation = sample_conversations[20]
evaluate_conversation(sample_conversation, verbose=True)

Runs available in conversation: [0, 1, 2]
*********************************************
Evaluating conversation run with ID: 0
Run ID: 0
Query: [{'role': 'system', 'content': 'You are an insurance claims processing agent. You help customers file claims, check claim statuses, and update claim details.'}, {'createdAt': 1741618732, 'role': 'user', 'content': [{'type': 'text', 'text': 'What is the current status of my claim number CLM12345?'}]}]
Response: [{'createdAt': 1741618737, 'run_id': '0', 'role': 'assistant', 'content': [{'type': 'text', 'text': 'I’m checking the status of your claim CLM12345 now. One moment please.'}]}]
Tool Definitions: [{'name': 'file_claim', 'description': 'File a new insurance claim.', 'parameters': {'type': 'object', 'properties': {'policy_number': {'type': 'string', 'description': 'The insurance policy number.'}, 'incident_description': {'type': 'string', 'description': 'Description of the incident.'}}}}, {'name': 'update_claim', 'description': 'Update detai

[{'intent_resolution': 2.0,
  'intent_resolution_result': 'fail',
  'intent_resolution_threshold': 5,
  'intent_resolution_reason': "The assistant's response acknowledges the user's query and indicates that it is checking the status of the claim. However, it does not provide any actionable information or resolution regarding the claim status, nor does it invoke any tools to retrieve the status. This leaves the query unresolved.",
  'additional_details': {'conversation_has_intent': True,
   'agent_perceived_intent': 'check the status of claim CLM12345',
   'actual_user_intent': 'check the status of claim CLM12345',
   'correct_intent_detected': True,
   'intent_resolved': False}},
 {'intent_resolution': 2.0,
  'intent_resolution_result': 'fail',
  'intent_resolution_threshold': 5,
  'intent_resolution_reason': "The assistant's response does not accurately address the user's follow-up query about their claim status. Instead, it invokes the 'file_claim' tool, which is unrelated to the use